# RVC AI Cover Maker (RVC Project) PT-BR
Criado por [ShiromiyaG](https://github.com/ShiromiyaG)
- Colab inspirado no [AICoverGen](https://github.com/SociallyIneptWeeb/AICoverGen) por [SociallyIneptWeeb](https://github.com/SociallyIneptWeeb)
- Usa a versão do [RVC-Project](https://github.com/RVC-Project) do [Retrieval-based-Voice-Conversion](https://github.com/RVC-Project/Retrieval-based-Voice-Conversion)

In [ ]:
import os
import subprocess
from google.colab import drive
from pathlib import Path
import requests
from IPython.display import clear_output
import subprocess
import fileinput
import zipfile
import shutil
!pip install python-dotenv
from dotenv import load_dotenv, set_key

drive.mount('/content/drive')
Path('/content/drive/MyDrive/RVC AI Cover Maker Input').mkdir(parents=True, exist_ok=True)
Path('/content/musicas/arquivos-originais').mkdir(parents=True, exist_ok=True)
os.chdir("/content")
# @title # Instalar
# @markdown ##### Sim, leva um tempo
# @markdown ##### Deixe os arquivos extras somente se for usar Enemble nos vocais
voc_extra_files = True #@param {type:"boolean"}
# @markdown ##### Para achar bugs, tem a opção de não limpar o output **(Pode causar crashes)**
not_clean_outputs = False #@param {type:"boolean"}
!sudo apt install ffmpeg python3.10-venv
!pip install poetry pedalboard
!poetry config virtualenvs.create false
Path('/content/arquivos').mkdir(parents=True, exist_ok=True)

%cd python-audio-separator
!pip install "audio-separator[gpu]" pyrubberband yt-dlp[default] wget git+https://github.com/IAHispano/gdown
if not_clean_outputs != True:
  clear_output()
%cd ../

if voc_extra_files == True:
  !git clone https://github.com/ZFTurbo/Music-Source-Separation-Training.git
  !wget -P /Music-Source-Separation-Training/models "https://github.com/TRvlvr/model_repo/releases/download/all_public_uvr_models/model_bs_roformer_ep_317_sdr_12.9755.ckpt"
  !wget -P /Music-Source-Separation-Training/models "https://github.com/ShiromiyaG/RVC-AI-Cover-Maker/releases/download/Dependencies/model_bs_roformer_ep_317_sdr_12.9755.yaml"
!wget -P ./arquivos "https://github.com/ShiromiyaG/RVC-AI-Cover-Maker/releases/download/Dependencies/spectograma.py" # Spectograma
!wget -P ./arquivos "https://github.com/ShiromiyaG/RVC-AI-Cover-Maker/releases/download/Dependencies/karokee_4band_v2_sn.pth" # karokee
!wget -P ./arquivos "https://github.com/ShiromiyaG/RVC-AI-Cover-Maker/releases/download/Dependencies/download_checks.json" # download_checks
!wget -P ./arquivos "https://github.com/ShiromiyaG/RVC-AI-Cover-Maker/releases/download/Dependencies/reverbpedalboard.py" # reverb_pedalboard
!wget -P ./arquivos "https://github.com/ShiromiyaG/RVC-AI-Cover-Maker/releases/download/Dependencies/utils.py" # utils
if not_clean_outputs != True:
  clear_output()
!git clone https://github.com/shirounanashi/OrpheusDL.git
%cd OrpheusDL
if not_clean_outputs != True:
  !poetry install --no-root -q > /dev/null 2>&1
  clear_output()
else:
  !poetry install --no-root
%cd ../
!git clone https://git.ovosimpatico.com/ovosimpatico/orpheusdl-deezer.git ./OrpheusDL/modules/deezer
!wget -P OrpheusDL/config "https://github.com/ShiromiyaG/RVC-AI-Cover-Maker/releases/download/Dependencies/settings.json"
if not_clean_outputs != True:
  clear_output()
!git clone https://github.com/RVC-Project/Retrieval-based-Voice-Conversion.git
%cd Retrieval-based-Voice-Conversion
if not_clean_outputs != True:
  !pip install rvc
  clear_output()
else:
  !poetry install --no-root
!rcv env create
load_dotenv()
set_key(".env", "rmvpe_root", "/content/Retrieval-based-Voice-Conversion")
set_key(".env", "hubert_path", "/content/Retrieval-based-Voice-Conversion/hubert_base.pt")
%cd ../
!wget -P ./Retrieval-based-Voice-Conversion "https://github.com/ShiromiyaG/RVC-AI-Cover-Maker/releases/download/Dependencies/rvc.py"
!wget -P ./Retrieval-based-Voice-Conversion "https://huggingface.co/IAHispano/Applio/resolve/main/Resources/hubert_base.pt"
!wget -P ./Retrieval-based-Voice-Conversion "https://huggingface.co/IAHispano/Applio/resolve/main/Resources/rmvpe.pt"

#!pip --disable-pip-version-check install beautifulsoup4 ffmpeg ffmpeg-python==0.1.18 numpy==1.23.5 requests==2.31.0 tqdm faiss-cpu==1.7.3 librosa==0.9.1 pydub==0.25.1 pyworld==0.3.4 praat-parselmouth==0.4.2 resampy==0.4.2 scipy==1.11.1 sounddevice==0.4.6 soundfile==0.12.1 torchaudio==2.1.1 praat-parselmouth noisereduce fairseq numba onnxruntime onnxruntime_gpu==1.15.1 torch==2.1.1 torchcrepe==0.0.21 torchgen>=0.0.1 torch_directml torchvision==0.16.1 einops local-attention matplotlib==3.7.2 tensorboard ffmpy==0.3.1 tensorboardX edge-tts==6.1.9 pydantic fastapi uvicorn
!pip uninstall onnxruntime-gpu -y
!python -m pip --disable-pip-version-check -q install onnxruntime-gpu --extra-index-url https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/onnxruntime-cuda-12/pypi/simple/
if not_clean_outputs != True:
  clear_output()

In [ ]:
#@title # Arquivo local
from google.colab import files
import os
import fnmatch
from pathlib import Path
Path('/content/musicas/arquivos-originais').mkdir(parents=True, exist_ok=True)
def search_files(pasta):
    arquivos = []
    for root, dirs, files in os.walk(pasta):
        for file in files:
            if file.endswith(".mp3") or file.endswith(".flac") or file.endswith(".wav"):
                caminho_arquivo = os.path.join(root, file)
                arquivos.append(caminho_arquivo)
    return arquivos
uploaded = files.upload()
uploaded = search_files("/content")
for arquivo in uploaded:
  !cp "{arquivo}" "/content/musicas/arquivos-originais"

In [ ]:
#@title # Arquivo do Google Drive
drive_path = "/content/drive/MyDrive/RVC AI Cover Maker Input" #@param {type:"string"}
import os
import fnmatch
from pathlib import Path
Path('/content/musicas/arquivos-originais').mkdir(parents=True, exist_ok=True)

def search_files(pasta):
    arquivos = []
    for root, dirs, files in os.walk(pasta):
        for file in files:
            if fnmatch.fnmatch(file, '*.mp3') or fnmatch.fnmatch(file, '*.flac') or fnmatch.fnmatch(file, '*.wav'):
                arquivos.append(os.path.join(root, file))
    return arquivos

arquivos_encontrados = search_files(drive_path)

for arquivo in arquivos_encontrados:
    !cp "{arquivo}" "/content/musicas/arquivos-originais"

In [ ]:
import os
from pathlib import Path
from IPython.display import Audio, display, clear_output
from glob import glob
from pydub import AudioSegment
from pathlib import Path
import json
from yt_dlp import YoutubeDL
import subprocess
import uuid
import gdown
import zipfile
import requests
from dotenv import load_dotenv
from scipy.io import wavfile
from rvc.modules.vc.modules import VC
# @title # Programa
Path('/content/musicas').mkdir(parents=True, exist_ok=True)
Path('/content/musicas/sem-intrumental').mkdir(parents=True, exist_ok=True)
Path("/content/drive/MyDrive/RVC AI Cover Maker").mkdir(parents=True, exist_ok=True)
input_folder = "/content/musicas/arquivos-originais"
no_back_folder = "/content/musicas/sem-back"
no_inst_folder = "/content/musicas/sem-intrumental"
output_folder = "/content/musicas/output-folder"
model_dir = "/content/arquivos"
stage1_dir = "/content/musicas/inst-etapa1"
stage2_dir = "/content/musicas/inst-etapa2"
final_output_dir = "/content/musicas/output_inst"
model_destination_folder = f"/content/RVC_CLI/logs"
output_drive = "/content/drive/MyDrive/RVC AI Cover Maker"
# @markdown ## Selecione uma das opções para fazer o donwload
link_of_yt = "" #@param {type:"string"}
link_of_deezer = "" #@param {type:"string"}
# @markdown ##### Se você vai usar o Deezer
bf_secret = "" #@param {type:"string"}
track_url_key = "" #@param {type:"string"}
arl = "" #@param {type:"string"}
# @markdown ## Selecione os algoritmos de Ensemble (junção de espectograma dos áudios)
# @markdown ##### Ensemble leva mais tempo, mas a qualidada final é maior
Vocals_Ensemble = True #@param {type:"boolean"}
algorithm_ensemble_vocals =  "averege" # @param ['averege', 'Max Spec', 'Min Spec'] {allow-input: false}
Instrumental_Ensemble = True #@param {type:"boolean"}
algorithm_ensemble_inst =  "Max Spec" # @param ['averege', 'Max Spec', 'Min Spec'] {allow-input: false}
# @markdown ## Configurações do RVC
rvc_model_link = ""  # @param {type:"string"}
rvc_model_name = os.path.splitext(os.path.basename(rvc_model_link))
f0method = "rmvpe"  # @param ["pm", "harvest", "crepe", "rmvpe"] {allow-input: false}
Pitch = 0  # @param {type:"slider", min:-24, max:24, step:0}
filter_radius = 3  # @param {type:"slider", min:0, max:10, step:0}
rms_mix_rate = 0.8  # @param {type:"slider", min:0.0, max:1.0, step:0.1}
protect = 0.5  # @param {type:"slider", min:0.0, max:0.5, step:0.1}
index_rate = 0.7  # @param {type:"slider", min:0.0, max:1.0, step:0.1}
# @markdown ## Reverb do áudio
REVERB_SIZE = 0.15  # @param {type:"slider", min:0.0, max:1.0, step:0.01}
REVERB_WETNESS = 0.2 # @param {type:"slider", min:0.0, max:1.0, step:0.01}
REVERB_DRYNESS = 0.8 # @param {type:"slider", min:0.0, max:1.0, step:0.01}
REVERB_DAMPING = 0.7 # @param {type:"slider", min:0.0, max:1.0, step:0.01}
# @markdown ## Opção de mixagem do áudio
Voc_Vol = -4 # @param {type:"slider", min:-10, max:10, step:0}
Inst_Vol = 0 # @param {type:"slider", min:-10, max:10, step:0}
remove_noise_from_RVC = True  # @param{type:"boolean"}
noise_db_limit = -30 # @param {type:"slider", min:-50, max:0, step:0.1}
save_only_as_mp3 = False  # @param{type:"boolean"}
# @markdown ##### Para achar bugs, tem a opção de não limpar o output
not_clean_outputs = False #@param {type:"boolean"}

def verify_if_folder_is_empty(caminho_pasta):
    if not os.listdir(caminho_pasta):
        raise ValueError("Você não enviou um arquivo de áudio!")

def get_last_modified_file(directory, filter=''):
  arquivos = glob(directory + "/*")
  if filter != '':
      arquivos = [arquivo for arquivo in arquivos if filter in arquivo]
  if arquivos:
      return max(arquivos, key=os.path.getmtime)
  else:
      return None

def find_files(directory, extensions):
  files = glob(f'{directory}/**/*{extensions}', recursive=True)
  return files[0]

def convert_to_mp3(input_file, output_file):
    audio = AudioSegment.from_wav(input_file)
    audio.export(output_file, format="mp3")

if link_of_deezer != "":
  with open('/content/OrpheusDL/config/settings.json', 'r') as file:
    data = json.load(file)

  data['modules']['deezer']['bf_secret'] = bf_secret
  data['modules']['deezer']['track_url_key'] = track_url_key
  data['modules']['deezer']['arl'] = arl

  with open('/content/OrpheusDL/config/settings.json', 'w') as file:
    json.dump(data, file, indent=4)

if rvc_model_link == "":
  raise ValueError("You didn't put a link to an RVC model")

if link_of_yt != "":
  !yt-dlp --extract-audio -o "/content/musicas/arquivos-originais/%(title)s.%(ext)s"  --audio-format mp3 --audio-quality 0 "{link_of_yt}"
  if not_clean_outputs != True:
    clear_output()
  arquivo = get_last_modified_file(input_folder)
  !cp "{arquivo}" "{input_folder}"
  print("Seu áudio já foi baixado!")

if link_of_deezer != "":
  if bf_secret != "" and track_url_key != "" and arl != "":
    %cd OrpheusDL
    !python orpheus.py "{link_of_deezer}"
    if not_clean_outputs != True:
      clear_output()
    %cd ../
    print("Seu áudio já foi baixado!")
else:
  raise ValueError("Você precisa fornecer a bf secret, track url key arl da sua conta Deezer Premium")

try:
    verify_if_folder_is_empty(input_folder)
except ValueError as e:
    print(e)

Path(input_folder).mkdir(parents=True, exist_ok=True)
Path(no_back_folder).mkdir(parents=True, exist_ok=True)
Path(no_inst_folder).mkdir(parents=True, exist_ok=True)
Path(output_folder).mkdir(parents=True, exist_ok=True)
Path(stage1_dir).mkdir(parents=True, exist_ok=True)
Path(stage2_dir).mkdir(parents=True, exist_ok=True)
Path(final_output_dir).mkdir(parents=True, exist_ok=True)
Path(output_drive).mkdir(parents=True, exist_ok=True)

input_files = glob(os.path.join(input_folder, '*.flac')) + glob(os.path.join(input_folder, '*.mp3')) + glob(os.path.join(input_folder, '*.wav'))
input_file_name = input_files[0]
for input_file in input_files:
  filename = os.path.basename(input_file)
  if filename.endswith(".mp3"):
    flac_filename = os.path.splitext(filename)[0] + '.flac'
    if not os.path.exists(flac_filename):
        audio = AudioSegment.from_mp3(input_file)
        audio.export(flac_filename, format="flac")
        os.remove(input_file)
        input_file = flac_filename
  subprocess.run([
      'audio-separator',
      f"{input_file}",
      '--model_file_dir', "/content/arquivos",
      '-m', 'MDX23C-8KFFT-InstVoc_HQ.ckpt',
      "--normalization", "1.0",
      "--output_format", "FLAC",
      "--single_stem", "Vocals",
      "--mdxc_batch_size", "8",
      '--output_dir', f"{no_inst_folder}"
  ])
  print(f"O processo de {filename} com MDX23C-8KFFT-InstVoc_HQ acabou!")
  if Vocals_Ensemble == True:
    lista = get_last_modified_file(no_inst_folder)
    subprocess.run([
        "python",
        "Music-Source-Separation-Training/inference.py",
       "--model_type", "bs_roformer",
        "--config_path", "Music-Source-Separation-Training/models/model_bs_roformer_ep_317_sdr_12.9755.yaml",
        "--start_check_point", "Music-Source-Separation-Training/models/model_bs_roformer_ep_317_sdr_12.9755.ckpt",
        "--input_folder", f"{input_file}",
        "--store_dir", f"{no_inst_folder}"
    ])
    print(f"O processo de {filename} com BSRoformer acabou!")
    lista = get_last_modified_file(no_inst_folder, "Vocals")
    ensemble_voc = os.path.join(no_inst_folder, f"{filename}_ensemble1.flac")
    subprocess.run([
        "python", "/content/arquivos/spectograma.py", "--audio_input", lista[0], lista[1],
        "--algorithm", f"{algorithm_ensemble_vocals}",
        "--is_normalization", "False",
        "--wav_type_set", "PCM_16",
        "--save_path", f"{ensemble_voc}"
    ])
  print(f"O processo de Ensemble de {filename} acabou!")
  filename = get_last_modified_file(no_inst_folder)
  no_inst_output = os.path.join(no_inst_folder, filename)
  subprocess.run([
      'audio-separator',
      f"{no_inst_output}",
      '--model_file_dir', "/content/arquivos",
      '-m', 'karokee_4band_v2_sn.pth',
      '--vr_window_size', '320',
      '--vr_aggression', '5',
      "--normalization", "1.0",
      "--vr_batch_size", "8",
      '--vr_enable_tta',
      "--output_format", "FLAC",
      "--single_stem", "Vocals",
      '--output_dir', f"{no_back_folder}"
  ])
  filename = get_last_modified_file(no_back_folder)
  no_back_output = os.path.join(no_back_folder, filename)
  print(f"O processamento de {filename} com o karokee_4band_v2_sn acabou!")
  subprocess.run([
      'audio-separator',
      f"{no_back_output}",
      '--model_file_dir', "/content/arquivos",
      '-m', 'UVR-DeEcho-DeReverb.pth',
      '--vr_window_size', '320',
      '--vr_aggression', '5',
      "--normalization", "1.0",
      "--vr_batch_size", "8",
      '--vr_enable_tta',
      "--output_format", "FLAC",
      "--single_stem", "No Reverb",
      '--output_dir', f"{output_folder}"
  ])
  print(f"O processamento de {filename} com o UVR-DeEcho-DeReverb acabou!")
print("Processamento doos vocais concluido.")

ensemble1_inputs = []

# Pass 1
for filename in os.listdir(input_folder):
  if filename.endswith(".flac") and filename.endswith(".mp3") and filename.endswith(".wav"):
      continue
  if filename.endswith(".mp3"):
    flac_filename = os.path.splitext(filename)[0] + '.flac'
    if not os.path.exists(flac_filename):
        audio = AudioSegment.from_mp3(filename)
        audio.export(flac_filename, format="flac")
        os.remove(filename)
        filename = flac_filename
  file_path = os.path.join(input_folder, filename)
  basename, _ = os.path.splitext(filename)
  if Instrumental_Ensemble == True:
    pass1_outputs = []
    processed_models = []
    model_names = ["5_HP-Karaoke-UVR.pth", "UVR-MDX-NET-Inst_HQ_4.onnx", "htdemucs.yaml"]
    for model_name in model_names:
        if model_name == "5_HP-Karaoke-UVR.pth":
            subprocess.run([
                "audio-separator",
                f"{file_path}",
                "--model_file_dir", f"{model_dir}",
                "-m", f"{model_name}",
                "--normalization", "1.0",
                "--vr_batch_size", "8",
                "--vr_enable_tta",
                "--output_format", "FLAC",
                "--single_stem", "Instrumental",
                "--output_dir", f"{stage1_dir}"
            ])
            processed_models.append(model_name)
            print(f"O processamento de {filename} com o {model_name} acabou!")
        if model_name == "UVR-MDX-NET-Inst_HQ_4.onnx":
            subprocess.run([
              "audio-separator",
              f"{file_path}",
              "--model_file_dir", f"{model_dir}",
              "-m", f"{model_name}",
              "--normalization", "1.0",
              "--mdx_batch_size", "8",
              "--output_format", "FLAC",
              "--single_stem", "Instrumental",
              "--output_dir", f"{stage1_dir}"
            ])
            processed_models.append(model_name)
            print(f"{filename} processing with {model_name} is over!")
        if model_name == "htdemucs.yaml":
            subprocess.run([
                "audio-separator",
                f"{file_path}",
                "--model_file_dir", f"{model_dir}",
                "-m", f"{model_name}",
                "--output_format", "FLAC",
                "--output_dir", f"{stage1_dir}"
            ])
            audio_files = [
                os.path.join(stage1_dir, f"{basename}_(Drums)_htdemucs.flac"),
                os.path.join(stage1_dir, f"{basename}_(Bass)_htdemucs.flac"),
                os.path.join(stage1_dir, f"{basename}_(Other)_htdemucs.flac")
            ]

            combined_audio = AudioSegment.from_file(audio_files[0], format="flac")

            for audio_file in audio_files[1:]:
                audio = AudioSegment.from_file(audio_file, format="flac")
                combined_audio = combined_audio.overlay(audio)

            combined_audio.export(f"{stage1_dir}/{basename}_demucs_(Instrumental).flac", format="flac")

            for audio_file in audio_files:
                os.remove(audio_file)
            processed_models.append(model_name)
            print(f"O processo de {basename} com {model_name} acabou!")
        model_names = [model for model in model_names if model not in processed_models]

  else:
    model_name = "UVR-MDX-NET-Inst_HQ_4.onnx"
    subprocess.run([
        "audio-separator",
        f"{file_path}",
        "--model_file_dir", f"{model_dir}",
        "-m", f"{model_name}",
        "--normalization", "1.0",
        "--mdx_batch_size", "8",
        "--output_format", "FLAC",
        "--single_stem", "Instrumental",
        "--output_dir", f"{stage1_dir}"
    ])
    final_output_path = os.path.join(stage1_dir, f"{basename}_model_name_without_ext.flac")
    print(f"O processo de {basename} com {model_name} acabou!")

if Instrumental_Ensemble == True:
  all_files = os.listdir(stage1_dir)
  pass1_outputs_filtered = [os.path.join(stage1_dir, output) for output in all_files if "Instrumental" in output]
  name = os.path.commonprefix(pass1_outputs_filtered)
  # First Ensemble
  ensemble1_output = os.path.join(stage1_dir, f"{name}_ensemble1.flac")
  result = subprocess.run([
      "python", "/content/arquivos/spectograma.py", "--audio_input", pass1_outputs_filtered[0], pass1_outputs_filtered[1], pass1_outputs_filtered[2],
      "--algorithm", f"{algorithm_ensemble_inst}",
      "--is_normalization", "False",
      "--wav_type_set", "PCM_16",
      "--save_path", f"{ensemble1_output}"
  ])

  print("O processo do primeiro Ensemble acabou!")

  # Pass 2
  basename = os.path.splitext(os.path.basename(ensemble1_output))[0]
  processed_models = []
  pass2_outputs = []
  model_names = ["karokee_4band_v2_sn.pth", "UVR-MDX-NET-Inst_HQ_4.onnx", "Kim_Vocal_2.onnx"]
  for model_name in model_names:
      if model_name == "karokee_4band_v2_sn.pth":
        model_name_without_ext = model_name.split('.')[0]
        output_path = os.path.join(stage2_dir, f"{basename}_(Instrumental)_{model_name_without_ext}.flac")
        pass2_outputs.append(output_path)
        subprocess.run([
            "audio-separator",
            f"{ensemble1_output}",
            "--model_file_dir", f"{model_dir}",
            "-m", model_name,
            "--vr_batch_size", "8",
            "--vr_enable_tta",
            "--normalization", "1.0",
            "--single_stem", "Instrumental",
            "--output_dir", f"{stage2_dir}"
        ])
        processed_models.append(model_name)
        print(f"Processamento de {basename} com o {model_name} acabou!")
      else:
        model_name_without_ext = model_name.split('.')[0]
        output_path = os.path.join(stage2_dir, f"{basename}_(Instrumental)_{model_name_without_ext}.flac")
        pass2_outputs.append(output_path)
        subprocess.run([
            "audio-separator",
            f"{ensemble1_output}",
            "--model_file_dir", f"{model_dir}",
            "-m", model_name,
            "--mdx_batch_size", "8",
            "--normalization", "1.0",
            "--single_stem", "Instrumental",
            "--output_dir", f"{stage2_dir}"
        ])
        processed_models.append(model_name)
        print(f"Processamento de {basename} com o {model_name} acabou!")
      model_names = [model for model in model_names if model not in processed_models]

  # Second Ensemble
  all_files = os.listdir(stage2_dir)
  pass2_outputs_filtered = [os.path.join(stage2_dir, output) for output in all_files if "Instrumental" in output]
  final_output_path = os.path.join(final_output_dir, f"{basename}_final_output.flac")
  subprocess.run([
      "python", "/content/arquivos/spectograma.py", "--audio_input", pass2_outputs_filtered[0], pass2_outputs_filtered[1], pass2_outputs_filtered[2],
      "--algorithm", f"{algorithm_ensemble_inst}",
      "--is_normalization", "False",
      "--wav_type_set", "PCM_16",
      "--save_path", f"{final_output_path}"
  ])
  print("Processing of the second Ensemble is over!")

if Instrumental_Ensemble == True:
  final_song = [output for output in final_output_path if "instrumental" in output]
else:
  final_song = [output for output in stage1_dir if "instrumental" in output]
print("Processamento dos instrumentais concluidos!")
name = os.path.commonprefix(final_song)
# RVC part
print("Fazendo download do modelo...")
filename = rvc_model_name
download_path = Path(model_destination_folder) / filename
if "drive.google.com" in rvc_model_link:
    gdown.download(rvc_model_link, str(download_path), quiet=False)
else:
    response = requests.get(rvc_model_link)
    with open(download_path, 'wb') as file:
        file.write(response.content)
if str(download_path).endswith(".zip"):
  Path(f'/content/Retrieval-based-Voice-Conversion/{rvc_model_name}').mkdir(parents=True, exist_ok=True)
  with zipfile.ZipFile(download_path, 'r') as zip_ref:
      zip_ref.extractall(f"/content/Retrieval-based-Voice-Conversion/{rvc_model_name}")
print("Download complete.")
current_dir = "/content/Retrieval-based-Voice-Conversion"
model_folder = os.path.join(current_dir, f"{rvc_model_name}")

if not os.path.exists(model_folder):
  raise FileNotFoundError(f"Model directory not found: {model_folder}")

files_in_folder = os.listdir(f"{model_destination_folder}/{rvc_model_name}")
pth_file = find_files(model_destination_folder, ".pth")
index_file = find_files(model_destination_folder, ".index")

if pth_file is None or index_file is None:
  raise FileNotFoundError("No model found.")

input_path = get_last_modified_file(final_output_path)
output_path = "/content/output_rvc.flac"
%cd Retrieval-based-Voice-Conversion
set_key(".env", "index_root", "/content/Retrieval-based-Voice-Conversion/{rvc_model_name}")
!python rvc.py --pth_file "{pth_file}" --index_rate "{index_rate}" --input_path "{input_path}" --pitch "{pitch}" --f0method "{f0method}" --filter_radius "{filter_radius}" --rms_mix_rate "{rms_mix_rate}" --protect "{protect}" --output_file "{output_path}"
%cd ../

# Reverb part
input_song = "/content/output_rvc.wav"
output_song = "{final_song}"
Path(output_folder).mkdir(parents=True, exist_ok=True)

!python /content/arquivos/reverbpedalboard.py -i "{input_song}" -rsize "{REVERB_SIZE}" -rwet "{REVERB_WETNESS}" -rdry "{REVERB_DRYNESS}" -rdamp "{REVERB_DAMPING}" -oformat "WAV"
output_path = f"/content/output_rvc_mixed.wav"
# Mix part
if remove_noise_from_RVC == True:
  audio = AudioSegment.from_file(output_path)
  db_limit = noise_db_limit
  silenced_audio = AudioSegment.silent(duration=len(audio))
  chunk_length = 100
  for i in range(0, len(audio), chunk_length):
      chunk = audio[i:i+chunk_length]
      if chunk.dBFS > db_limit:
          silenced_audio = silenced_audio.overlay(chunk, position=i)
  silenced_audio.export(output_path, format="wav")
output_mix = f"/content/{name}_{rvc_model_name}.flac"

!python /content/arquivos/utils.py --audio_paths  "{output_path}" "{output_song}" --output_path "{output_mix}" --main_gain "{Voc_Vol}" --inst_gain "{Inst_Vol}" --output_format "FLAC"

convert_to_mp3(output_mix, name)

if not_clean_outputs != True:
  clear_output()
if save_only_as_mp3 != True:
  !cp "{output_mix}" "{output_drive}"
  print("O arquivo FLAC está disponível na pasta \"RVC AI Cover Maker\" do seu Drive!")
  print("E o MP3 no player abaixo!\n")
else:
  !cp "{name}" "{output_drive}"
  print("O arquivo MP3 está disponível na pasta \"RVC AI Cover Maker\" do seu Drive!")
  print("E no player abaixo!\n")
display(Audio(name, autoplay = False))